## Workflow 
##### (especificado no Canvas pelo prof. Leandro)

- Baixar os dados do MovieLens 100k (uma base contendo informações sobre filmes assistidos e avaliados por vários usuários)
- Rodar o algoritmo de filtragem colaborativa baseada no usuário (basicamente o algoritmo KNN) nesses dados. Para isso você pode usar a biblioteca [surpriselib](http://surpriselib.com/). [Aqui](http://surprise.readthedocs.io/en/stable/knn_inspired.html) você tem os algoritmos baseados em KNN que pode usar.
- O que devo entregar?
    - Link para código comentado no github.
    - Qualidade do modelo em termos de RMSE (veja seção Getting Started na página [http://surpriselib.com/](http://surpriselib.com/))
    - Uma demo web onde você passar um usuário, mostra as top-5 recomendaçõe e apresenta os três melhores vizinhos desse usuário)


## Solução